In [1]:
import pandas as pd
from tqdm import tqdm
import pickle

# Datos sin correccion monetaria

In [2]:
years = [2014, 2015, 2016, 2017, 2018, 2019]

df = pd.DataFrame()

pbar = tqdm(total=len(years))
for i, year in enumerate(years):
    # carga de datos, skiprows=1 porque hay una fila con el año
    _df = pd.read_excel(f"../Datos/02_intermediate/sinim/sinim_{year}.xlsx", engine="openpyxl", skiprows=1)

    # cambio de nombre a las primeras dos columnas que corresponden al codigo y municipio
    _df.rename(mapper={"Unnamed: 0":"codigo", "Unnamed: 1":"municipio"}, axis=1, inplace=True)
    
    # se elimina la primera fila pues tiene solo el año
    _df.drop(0, inplace=True)

    # crear un glosario de datos
    if i==0:
        col_name_description = {col.split(" ")[0]: col.replace(col.split(" ")[0], "") for col in _df.columns if not (col=="codigo" or col=="municipio")}
        with open("../Datos/09_dictionaries/glosario_columnas_sinim.pkl", "wb") as file:
            pickle.dump(col_name_description, file)
    
    # cambiar nombres de las columnas
    col_name_mapper = {col: col.split(" ")[0] for col in _df.columns if not (col=="codigo" or col=="municipio")}
    _df.rename(mapper=col_name_mapper, axis=1, inplace=True)

    # transformar a datos numéricos
    for col in df.columns:
        if not(col=="codigo" or col=="municipio" or col=="año"):
            if "(S-N)" not in col_name_description[col]:
                _df[col] = pd.to_numeric(_df[col], errors="coerce")

    # agregar año
    _df["año"] = year
    
    # concatenar
    df = pd.concat([df, _df], axis=0)
    pbar.update(1)

pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:05<00:00,  1.13it/s]


In [3]:
df.reset_index(drop=True)
df.to_csv(f"../Datos/03_primary/sinim_{min(years)}_{max(years)}_consolidado.csv")

In [4]:
df

,codigo,municipio,BPIGM,BPISICMUN,BPIIM,BPVGM,BPVMSALINI,BPVIM,IADM83,IADM97,...,ITPCF,ITPCMA,ITPR,ITPU,ICAR005,ICAR006,ICAR008,ICAR015,ICAR014,año
1,1101,IQUIQUE,29773534.0,No Recepcionado,29773534.0,54852651.0,10034406.0,54852651.0,0.0,959868.0,...,96471,99966,No Recepcionado,No Recepcionado,49.11,50.89,No Recepcionado,8.32,17.48,2014
2,1107,ALTO HOSPICIO,8475076.0,No Recepcionado,8475075.0,12562279.0,201592.0,12562273.0,0.0,78971.0,...,52344,53734,No Recepcionado,No Recepcionado,49.34,50.66,No Recepcionado,4.53,20.99,2014
3,1401,POZO ALMONTE,5237503.0,No Recepcionado,5237503.0,6765437.0,10.0,6765432.0,0.0,5622.0,...,5383,8340,No Recepcionado,No Recepcionado,39.23,60.77,No Recepcionado,12.35,18.33,2014
4,1402,CAMIÑA,1079751.0,No Recepcionado,1079251.0,2592401.0,891151.0,2592399.0,0.0,256.0,...,593,704,No Recepcionado,No Recepcionado,45.72,54.28,No Recepcionado,0.00,9.24,2014
5,1403,COLCHANE,2589313.0,No Recepcionado,2589312.0,2881817.0,185000.0,2881815.0,0.0,1020.0,...,768,930,No Recepcionado,No Recepcionado,45.23,54.77,No Recepcionado,0.00,1.77,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,16301,SAN CARLOS,10459691.0,1.2e+06,10459701.0,14039478.0,2717805.0,14039487.0,0.0,172218.0,...,28826,27109,19763,33261,51.53,48.47,37.27,8.61,10.51,2019
342,16302,COIHUECO,5001190.0,311840,5001190.0,9005407.0,1891290.0,9005407.0,0.0,27558.0,...,14319,13828,17783,9098,50.87,49.13,66.15,12.42,11.63,2019
343,16303,ÑIQUÉN,3386698.0,250000,3386698.0,3616226.0,250000.0,3616226.0,0.0,2693.0,...,5827,5729,9999,1153,50.42,49.58,89.66,12.20,7.11,2019
344,16304,SAN FABIÁN,1798500.0,23000,1798500.0,2721631.0,473118.0,2721629.0,0.0,7802.0,...,2341,2266,2614,1694,50.81,49.19,60.68,0.00,9.97,2019


# Datos con correccion monetaria

In [5]:
years = [2014, 2015, 2016, 2017, 2018, 2019]

df = pd.DataFrame()

pbar = tqdm(total=len(years))
for i, year in enumerate(years):
    # carga de datos, skiprows=1 porque hay una fila con el año
    _df = pd.read_excel(f"../Datos/02_intermediate/sinim/sinim_{year}_correccion.xlsx", engine="openpyxl", skiprows=1)

    # cambio de nombre a las primeras dos columnas que corresponden al codigo y municipio
    _df.rename(mapper={"Unnamed: 0":"codigo", "Unnamed: 1":"municipio"}, axis=1, inplace=True)
    
    # se elimina la primera fila pues tiene solo el año
    _df.drop(0, inplace=True)

    # crear un glosario de datos
    if i==0:
        col_name_description = {col.split(" ")[0]: col.replace(col.split(" ")[0], "") for col in _df.columns if not (col=="codigo" or col=="municipio")}
        with open("../Datos/09_dictionaries/glosario_columnas_sinim.pkl", "wb") as file:
            pickle.dump(col_name_description, file)
    
    # cambiar nombres de las columnas
    col_name_mapper = {col: col.split(" ")[0] for col in _df.columns if not (col=="codigo" or col=="municipio")}
    _df.rename(mapper=col_name_mapper, axis=1, inplace=True)

    # transformar a datos numéricos
    for col in df.columns:
        if not(col=="codigo" or col=="municipio" or col=="año"):
            if "(S-N)" not in col_name_description[col]:
                _df[col] = pd.to_numeric(_df[col], errors="coerce")

    # agregar año
    _df["año"] = year
    
    # concatenar
    df = pd.concat([df, _df], axis=0)
    pbar.update(1)

pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.20it/s]


In [6]:
df.reset_index(drop=True)
df.to_csv(f"../Datos/03_primary/sinim_con_correccion_{min(years)}_{max(years)}_consolidado.csv")

In [7]:
df

,codigo,municipio,BPIGM,BPISICMUN,BPIIM,BPVGM,BPVMSALINI,BPVIM,IADM83,IADM97,...,ITPCF,ITPCMA,ITPR,ITPU,ICAR005,ICAR006,ICAR008,ICAR015,ICAR014,año
1,1101,IQUIQUE,42933436.0,No Recepcionado,42933436.0,79097523.0,14469613.0,79097523.0,0.0,1384130.0,...,96471,99966,No Recepcionado,No Recepcionado,49.11,50.89,No Recepcionado,8.32,17.48,2014
2,1107,ALTO HOSPICIO,12221060.0,No Recepcionado,12221058.0,18114806.0,290696.0,18114798.0,0.0,113876.0,...,52344,53734,No Recepcionado,No Recepcionado,49.34,50.66,No Recepcionado,4.53,20.99,2014
3,1401,POZO ALMONTE,7552479.0,No Recepcionado,7552479.0,9755760.0,14.0,9755753.0,0.0,8107.0,...,5383,8340,No Recepcionado,No Recepcionado,39.23,60.77,No Recepcionado,12.35,18.33,2014
4,1402,CAMIÑA,1557001.0,No Recepcionado,1556280.0,3738242.0,1285040.0,3738239.0,0.0,369.0,...,593,704,No Recepcionado,No Recepcionado,45.72,54.28,No Recepcionado,0.00,9.24,2014
5,1403,COLCHANE,3733789.0,No Recepcionado,3733788.0,4155580.0,266770.0,4155577.0,0.0,1471.0,...,768,930,No Recepcionado,No Recepcionado,45.23,54.77,No Recepcionado,0.00,1.77,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,16301,SAN CARLOS,13022315.0,1.494e+06,13022328.0,17479150.0,3383667.0,17479161.0,0.0,214411.0,...,28826,27109,19763,33261,51.53,48.47,37.27,8.61,10.51,2019
342,16302,COIHUECO,6226482.0,388241,6226482.0,11211732.0,2354656.0,11211732.0,0.0,34310.0,...,14319,13828,17783,9098,50.87,49.13,66.15,12.42,11.63,2019
343,16303,ÑIQUÉN,4216439.0,311250,4216439.0,4502201.0,311250.0,4502201.0,0.0,3353.0,...,5827,5729,9999,1153,50.42,49.58,89.66,12.20,7.11,2019
344,16304,SAN FABIÁN,2239133.0,28635,2239133.0,3388431.0,589032.0,3388428.0,0.0,9713.0,...,2341,2266,2614,1694,50.81,49.19,60.68,0.00,9.97,2019
